In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

C:\Users\Jinbao\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
train_data = pd.read_csv('data/dataTrain.csv')
test_data = pd.read_csv('data/dataA.csv')
submission = pd.read_csv('data/submit_example_A.csv')
data_nolabel = pd.read_csv('data/dataNoLabel.csv')
print(f'train_data.shape = {train_data.shape}')
print(f'test_data.shape  = {test_data.shape}')
print(f'data_nolabel.shape  = {data_nolabel.shape}')

train_data.shape = (59872, 48)
test_data.shape  = (49858, 47)
data_nolabel.shape  = (39884, 47)


In [3]:
def func3(x, y):
    if x<y:
        return int(x/3)
    return int(y/3)

In [4]:
for col in ['f%d' %idx for idx in range(4, 9)]:
    train_data[col] = train_data[col].apply(lambda x: func3(x, 102))
    test_data[col] = test_data[col].apply(lambda x: func3(x, 102))
for col in ['f%d' %idx for idx in range(9, 14)]:
    train_data[col] = train_data[col].apply(lambda x: func3(x, 30))
    test_data[col] = test_data[col].apply(lambda x: func3(x, 30))
for col in ['f%d' %idx for idx in range(14, 19)]:
    train_data[col] = train_data[col].apply(lambda x: func3(x, 3))
    test_data[col] = test_data[col].apply(lambda x: func3(x, 3))

train_data['f19'] = train_data['f19'].apply(lambda x: func3(x, 30))
test_data['f19'] = test_data['f19'].apply(lambda x: func3(x, 30))

for col in ['f%d' %idx for idx in range(20, 42)]:
    train_data[col] = train_data[col].apply(lambda x: func3(x, 300))
    test_data[col] = test_data[col].apply(lambda x: func3(x, 300))

In [4]:
train_data['label'].value_counts()

0    44950
1    14922
Name: label, dtype: int64

In [5]:
train_data.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f38,f39,f40,f41,f42,f43,f44,f45,f46,label
0,81167,0,1,mid,0,0,0,153,0,0,...,0,0,0,0,0,0,0,624,1539,0
1,50408,1,1,mid,0,0,21,0,0,0,...,0,0,0,0,0,0,0,186,366,0
2,9114,0,0,high,36,36,120,0,0,0,...,0,0,0,0,0,0,0,24,48,1
3,53228,1,1,low,0,0,0,0,0,0,...,0,0,0,0,0,0,3,3,9,0
4,56280,1,1,mid,9,51,294,0,0,0,...,0,0,0,0,0,0,0,42,141,0


In [6]:
train_data['f47'] = train_data['f1'] * 10 + train_data['f2']
test_data['f47'] = test_data['f1'] * 10 + test_data['f2']

train_data['f48'] = train_data.apply(lambda x: "%02d%s" %(x['f47'], x['f3']), axis=1)
test_data['f48'] = test_data.apply(lambda x: "%02d%s" %(x['f47'], x['f3']), axis=1)
# 暴力Feature 位置
loc_f = ['f1', 'f2', 'f4', 'f5', 'f6']
for df in [train_data, test_data]:
    for i in range(len(loc_f)):
        for j in range(i + 1, len(loc_f)):
            df[f'{loc_f[i]}+{loc_f[j]}'] = df[loc_f[i]] + df[loc_f[j]]
            df[f'{loc_f[i]}-{loc_f[j]}'] = df[loc_f[i]] - df[loc_f[j]]
            df[f'{loc_f[i]}*{loc_f[j]}'] = df[loc_f[i]] * df[loc_f[j]]
            df[f'{loc_f[i]}/{loc_f[j]}'] = df[loc_f[i]] / (df[loc_f[j]]+0.000001)

# 暴力Feature 通话
com_f = ['f43', 'f44', 'f45', 'f46']
for df in [train_data, test_data]:
    for i in range(len(com_f)):
        for j in range(i + 1, len(com_f)):
            df[f'{com_f[i]}+{com_f[j]}'] = df[com_f[i]] + df[com_f[j]]
            df[f'{com_f[i]}-{com_f[j]}'] = df[com_f[i]] - df[com_f[j]]
            df[f'{com_f[i]}*{com_f[j]}'] = df[com_f[i]] * df[com_f[j]]
            df[f'{com_f[i]}/{com_f[j]}'] = df[com_f[i]] / (df[com_f[j]]+0.000001)

In [7]:
cat_columns = ['f3', 'f48']
data = pd.concat([train_data, test_data])

for col in cat_columns:
    lb = LabelEncoder()
    lb.fit(data[col])
    train_data[col] = lb.transform(train_data[col])
    test_data[col] = lb.transform(test_data[col])

In [8]:
for col in [f'f{idx}' for idx in range(1, 49)]:
    tmp = train_data.groupby([col]).label.agg(['count'])
    tmp.columns = [f"{col}_{cols}" for cols in tmp.columns]
    tmp.reset_index(inplace=True)
    train_data = train_data.merge(tmp, on=[col], how="left")
    test_data = test_data.merge(tmp, on=[col], how="left")

In [9]:
train_data.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f39_count,f40_count,f41_count,f42_count,f43_count,f44_count,f45_count,f46_count,f47_count,f48_count
0,81167,0,1,2,0,0,0,153,0,0,...,58935,59866,59868,59532,39430,31672,39,8,24346,8562
1,50408,1,1,2,0,0,21,0,0,0,...,58935,59866,59868,59532,39430,31672,244,139,16711,6817
2,9114,0,0,0,36,36,120,0,0,0,...,58935,59866,59868,59532,39430,31672,799,287,17179,6031
3,53228,1,1,1,0,0,0,0,0,0,...,58935,59866,59868,59532,39430,4916,2287,838,16711,5479
4,56280,1,1,2,9,51,294,0,0,0,...,58935,59866,59868,59532,39430,31672,606,204,16711,6817


In [10]:
num_columns = [ col for col in train_data.columns if col not in ['id', 'label', 'f3', 'f48']]
feature_columns = num_columns + cat_columns
target = 'label'

train = train_data[feature_columns]
label = train_data[target]
test = test_data[feature_columns]
train = train[:50000]
label = label[:50000]

In [11]:
print(train.shape, label.shape)

(50000, 160) (50000,)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    train, label, stratify=label, random_state=2022)

In [13]:
gbm = LGBMClassifier(
    objective='binary',
    boosting_type='gbdt',
    num_leaves=2 ** 6, 
    max_depth=8,
    colsample_bytree=0.8,
    subsample_freq=1,
    max_bin=255,
    learning_rate=0.05, 
    n_estimators=100, 
    metrics='auc'
)

In [14]:
gbm.fit(X_train, y_train)
y_pred = gbm.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)
print('auc = %.8f' % auc)

auc = 0.91246765


In [15]:
features = []
feature_importances = []
for col in feature_columns:
    x_test = X_test.copy()
    x_test[col] = 0
    auc1 = roc_auc_score(y_test, gbm.predict_proba(x_test)[:, 1])
    if auc1 < auc:
        features.append(col)
    feature_importances.append([col, auc1, auc1 - auc])

In [16]:
feature_importances.sort(key=lambda x: x[2])
for fi in feature_importances:
    print("| %10s | %.8f | %.8f |" % (fi[0], fi[1], fi[2]))

|  f37_count | 0.90923563 | -0.00323202 |
|      f2-f4 | 0.90928619 | -0.00318146 |
|  f30_count | 0.90947590 | -0.00299175 |
|      f2/f6 | 0.91033904 | -0.00212861 |
|      f1-f4 | 0.91057496 | -0.00189269 |
|        f46 | 0.91068608 | -0.00178157 |
|        f48 | 0.91101849 | -0.00144916 |
|      f5-f6 | 0.91114610 | -0.00132155 |
|      f4/f5 | 0.91122128 | -0.00124636 |
|      f1/f6 | 0.91127701 | -0.00119064 |
|  f36_count | 0.91128870 | -0.00117895 |
|  f48_count | 0.91129707 | -0.00117058 |
|    f45/f46 | 0.91131852 | -0.00114913 |
|  f18_count | 0.91148150 | -0.00098615 |
|  f19_count | 0.91151958 | -0.00094807 |
|  f33_count | 0.91164657 | -0.00082108 |
|      f2/f5 | 0.91169527 | -0.00077238 |
|  f25_count | 0.91178680 | -0.00068085 |
|      f5/f6 | 0.91189356 | -0.00057409 |
|  f35_count | 0.91194169 | -0.00052596 |
|  f21_count | 0.91195895 | -0.00050870 |
|      f2/f4 | 0.91198250 | -0.00048515 |
|        f19 | 0.91205227 | -0.00041538 |
|   f8_count | 0.91205626 | -0.000

In [17]:
gbm.fit(X_train[features], y_train)
y_pred = gbm.predict_proba(X_test[features])[:, 1]
auc = roc_auc_score(y_test, y_pred)
print('auc = %.8f' % auc)

auc = 0.91312514


In [18]:
train = train[features]
test = test[features]

In [19]:
def model_train(model, model_name, kfold=5):
    oof_preds = np.zeros((train.shape[0]))
    test_preds = np.zeros(test.shape[0])
    skf = StratifiedKFold(n_splits=kfold, shuffle=True)
    print(f"Model = {model_name}")
    for k, (train_index, test_index) in enumerate(skf.split(train, label)):
        x_train, x_test = train.iloc[train_index, :], train.iloc[test_index, :]
        y_train, y_test = label.iloc[train_index], label.iloc[test_index]

        model.fit(x_train,y_train)

        y_pred = model.predict_proba(x_test)[:,1]
        oof_preds[test_index] = y_pred.ravel()
        auc = roc_auc_score(y_test,y_pred)
        print("- KFold = %d, val_auc = %.4f" % (k, auc))
        test_fold_preds = model.predict_proba(test)[:, 1]
        test_preds += test_fold_preds.ravel()
        
#         val_fold_preds = model.predict_proba(val)[:, 1]
#         val_preds += val_fold_preds.ravel()
    print("Overall Model = %s, AUC = %.4f" % (model_name, roc_auc_score(label, oof_preds)))
    return test_preds / kfold #, val_preds / kfold

In [20]:
test_preds = model_train(gbm, 'LGBMClassifier', 10)

Model = LGBMClassifier
- KFold = 0, val_auc = 0.9088
- KFold = 1, val_auc = 0.9187
- KFold = 2, val_auc = 0.9122
- KFold = 3, val_auc = 0.9083
- KFold = 4, val_auc = 0.9153
- KFold = 5, val_auc = 0.9063
- KFold = 6, val_auc = 0.9009
- KFold = 7, val_auc = 0.9082
- KFold = 8, val_auc = 0.9070
- KFold = 9, val_auc = 0.9081
Overall Model = LGBMClassifier, AUC = 0.9093


In [21]:
submission['label'] = test_preds
submission.to_csv('submission.csv', index=False)

In [22]:
gbc = GradientBoostingClassifier(
    n_estimators=50, 
    learning_rate=0.1,
    max_depth=5
)
hgbc = HistGradientBoostingClassifier(
    max_iter=100,
    max_depth=5
)
xgbc = XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    n_estimators=100, 
    max_depth=6, 
    learning_rate=0.1
)
gbm = LGBMClassifier(
    objective='binary',
    boosting_type='gbdt',
    num_leaves=2 ** 6, 
    max_depth=8,
    colsample_bytree=0.8,
    subsample_freq=1,
    max_bin=255,
    learning_rate=0.05, 
    n_estimators=100, 
    metrics='auc'
)
cbc = CatBoostClassifier(
    iterations=210, 
    depth=6, 
    learning_rate=0.03, 
    l2_leaf_reg=1, 
    loss_function='Logloss', 
    verbose=0
)


In [23]:
estimators = [
    ('gbc', gbc),
    ('hgbc', hgbc),
    ('xgbc', xgbc),
    ('gbm', gbm),
    ('cbc', cbc)
]
clf = StackingClassifier(
    estimators=estimators, 
    final_estimator=LogisticRegression()
)


In [24]:
clf_test_preds = model_train(clf, "StackingClassifier", 10)

Model = StackingClassifier
- KFold = 0, val_auc = 0.9114


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [18]:
test[target] = test_preds
val[target] = val_preds

In [26]:
testp = test[ (test[target]<=0.01) | (test[target]>=0.99) ].copy()
testp.loc[ testp[target]>=0.5, target ] = 1
testp.loc[ testp[target]<0.5, target ] = 0

valp = val[ (val[target]<=0.01) | (val[target]>=0.99) ].copy()
valp.loc[ valp[target]>=0.5, target ] = 1
valp.loc[ valp[target]<0.5, target ] = 0

trainp = pd.concat([train, testp[ff], valp[ff]])
labelp = pd.concat([label, testp[target], valp[target]])
test = test[ff]

In [27]:
print(trainp.shape, labelp.shape, test.shape)

(69513, 71) (69513,) (49858, 71)


In [8]:
preds = model_train(gbm, 'gbm', 10)

Model = gbm
- KFold = 0, val_auc = 0.9066
- KFold = 1, val_auc = 0.9083
- KFold = 2, val_auc = 0.9156
- KFold = 3, val_auc = 0.9043
- KFold = 4, val_auc = 0.9065
- KFold = 5, val_auc = 0.9070
- KFold = 6, val_auc = 0.8993
- KFold = 7, val_auc = 0.9160
- KFold = 8, val_auc = 0.9155
- KFold = 9, val_auc = 0.9144
Overall Model = gbm, AUC = 0.9092


In [18]:
train.shape, test.shape, label.shape

((50000, 111), (49858, 111), (50000,))

In [21]:
import gc
import joblib
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten, Input, Embedding, BatchNormalization
import warnings
warnings.filterwarnings("ignore")

In [22]:
def create_model(input_shape): 
    inputs = layers.Input(shape=(input_shape,))
    
    x = layers.BatchNormalization()(inputs)
    
    x = layers.Dense(500, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(2, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=y)
    return model

In [23]:
oof_preds = np.zeros((trainp.shape[0]))
test_preds = np.zeros(test.shape[0])

input_shape = trainp.shape[1]

In [31]:
%%time
skf = StratifiedKFold(n_splits=10, shuffle=True)
k = 0
for k, (train_index, test_index) in enumerate(skf.split(trainp, labelp)):
    K.clear_session()
    x_train, x_test = trainp.iloc[train_index, :], trainp.iloc[test_index, :]
    y_train, y_test = labelp.iloc[train_index], labelp.iloc[test_index]
    
    model = create_model(input_shape)
    adam = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[tf.keras.metrics.AUC()])
    lr = callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.1,
                                     patience=3, min_lr=1e-6, mode='max', verbose=0)
    es = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=5,
                                 verbose=0, mode='max', baseline=None, restore_best_weights=True)
    model.fit(x_train,
              utils.to_categorical(y_train),
              validation_data=(x_test, utils.to_categorical(y_test)),
              verbose=0,
              batch_size=2048,
              callbacks=[lr, es],
              epochs=100
             )
    valid_fold_preds = model.predict(x_test)[:, 1]
    test_fold_preds = model.predict(test)[:, 1]
    oof_preds[test_index] = valid_fold_preds.ravel()
    test_preds += test_fold_preds.ravel()
    print("- KFold = %d, val_auc = %.4f" % (k, metrics.roc_auc_score(y_test, valid_fold_preds)))
    k += 1
    K.clear_session()

1559/1559 [==============================] - 1s 843us/step
- KFold = 0, val_auc = 0.9390
1559/1559 [==============================] - 2s 1ms/step
- KFold = 1, val_auc = 0.9400
1559/1559 [==============================] - 1s 836us/step
- KFold = 2, val_auc = 0.9421
1559/1559 [==============================] - 1s 943us/step
- KFold = 3, val_auc = 0.9339
1559/1559 [==============================] - 1s 838us/step
- KFold = 4, val_auc = 0.9340
1559/1559 [==============================] - 1s 893us/step
- KFold = 5, val_auc = 0.9286
1559/1559 [==============================] - 1s 865us/step
- KFold = 6, val_auc = 0.9346
1559/1559 [==============================] - 1s 897us/step
- KFold = 7, val_auc = 0.9372
1559/1559 [==============================] - 2s 991us/step
- KFold = 8, val_auc = 0.9379
1559/1559 [==============================] - 1s 874us/step
- KFold = 9, val_auc = 0.9383
CPU times: total: 23min 8s
Wall time: 4min 46s


In [32]:
print("Overall AUC={}".format(metrics.roc_auc_score(labelp.values, oof_preds)))

Overall AUC=0.9362616597524223


In [35]:
submission['label'] = test_preds / 10
submission.to_csv('submission.csv', index=False)

In [36]:
submission.head()

,id,label
0,1,0.021979
1,2,0.039847
2,3,0.065599
3,4,0.051095
4,5,0.025593
